In [11]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import halo_data as hd


In [12]:
data = hd.getdata("C:/Users/LV/OneDrive - University of Helsinki/FMI/halo/53/depolarization/")
# data = hd.getdata("G:/OneDrive - University of Helsinki/FMI/halo/53/depolarization/")
# data = hd.getdata(r'G:\OneDrive - University of Helsinki\FMI\halo\53\depolarization')


In [16]:
file_name = next(data)
df = hd.halo_data(file_name)
# df.info
# df.full_data
# df.full_data_names
# df.data
# # Names of data
# df.data_names
# # More info
# df.more_info
# # Get meta data of each variable
# df.meta_data('co_signal')

# Get meta data of all variables
{'==>' + key: df.meta_data(key) for key in df.full_data_names}

# Only crucial info
{'==>' + key: df.meta_data(key)['_attributes'] for key in df.full_data_names}


In [17]:
# Change masking missing values from -999 to NaN
df.unmask999()
# Remove first three columns of data matrix due to calibration,
# they correspond to top 3 height data
# ask Ville for more detail
df.filter_height()
# Overview of data
df.describe()


,beta_raw,v_raw,v_error,co_signal,cross_signal,depo_raw,co_signal_averaged,cross_signal_averaged,depo_averaged_raw
count,1.6864e+06,1.6864e+06,8.6517e+05,1.6864e+06,1.656e+06,1.656e+06,1.5089e+05,1.5089e+05,1.5089e+05
mean,5.0269e-07,-0.37469,0.63624,1.0136,1.001,0.02717,1.0136,1.001,0.082565
std,4.0055e-06,3.7625,0.31674,0.14362,0.0093658,853.17,0.13009,0.0084732,199.15
min,-1.1076e-05,-18.946,0.0037261,0.99282,0.9936,-9.0147e+05,0.99763,0.99804,-45894
25%,-2.9248e-07,-1.8231,0.44862,0.99921,0.99916,-0.86385,0.99978,0.99977,-0.81253
50%,2.5706e-08,-0.5236,0.65611,1.0002,1.0001,0.050793,1.0001,1.0001,0.055489
75%,5.3697e-07,1.0435,0.87974,1.0013,1.0011,0.87428,1.0004,1.0004,0.82714
95%,2.9976e-06,6.7383,1.1116,1.0182,1.0032,5.7556,1.0206,1.0016,5.5614
max,0.00019629,18.816,1.1819,16.162,2.0177,3.5967e+05,12.348,1.7416,29554
Missing values,0,0,8.2127e+05,0,30432,30432,0,0,0


In [18]:
# Plot data
%matplotlib qt
df.plot(
    variables=['beta_raw', 'v_raw', 'cross_signal', 'depo_raw', 'co_signal',
               'cross_signal_averaged', 'depo_averaged_raw', 'co_signal_averaged'], ncol=2, size=(20, 15))


C:\Users\LV\Desktop\Python\halo-lidar\halo_data.py:54: RuntimeWarning: invalid value encountered in log10
  val = np.log10(self.data.get(var)).transpose()


In [20]:
%matplotlib qt
fig, ax = plt.subplots(1, 2, figsize=(24, 12))
p = ax[0].pcolormesh(df.data['time'],
                     df.data['range'],
                     df.data['co_signal'].transpose(),
                     cmap='jet', vmin=0.995, vmax=1.005)
area = hd.area_select(df.data['time'],
                      df.data['range'],
                      df.data['co_signal'].transpose(),
                      ax[0],
                      ax[1],
                      type='kde')
fig.colorbar(p, ax=ax[0])

Chosen 381720 values


In [21]:
noise = area.area - 1
threshold = 1 + np.nanstd(noise) * 3

np.nanstd(noise)
threshold


1.0041324748890474

In [22]:
%matplotlib qt
fig, ax = plt.subplots(1, 2, figsize=(24, 12))
p = ax[0].pcolormesh(df.data['time_averaged'],
                     df.data['range'],
                     df.data['co_signal_averaged'].transpose(),
                     cmap='jet', vmin=0.995, vmax=1.005)
area = hd.area_select(df.data['time_averaged'],
                      df.data['range'],
                      df.data['co_signal_averaged'].transpose(),
                      ax[0],
                      ax[1],
                      type='kde')
fig.colorbar(p, ax=ax[0])

Chosen 28122 values


In [23]:
noise_averaged = area.area - 1
threshold_averaged = 1 + np.nanstd(noise_averaged) * 2

threshold_averaged



1.000830159871839

In [24]:
df.filter(variables=['beta_raw', 'v_raw', 'cross_signal', 'depo_raw'],
          ref='co_signal', threshold=threshold)

df.filter(variables=['cross_signal_averaged', 'depo_averaged_raw'],
          ref='co_signal_averaged', threshold=threshold_averaged)


In [25]:
# Plot data
%matplotlib qt
df.plot(
    variables=['beta_raw', 'v_raw', 'cross_signal', 'depo_raw', 'co_signal',
               'cross_signal_averaged', 'depo_averaged_raw', 'co_signal_averaged'], ncol=2, size=(20, 15))



In [26]:
df.describe()


,beta_raw,v_raw,v_error,co_signal,cross_signal,depo_raw,co_signal_averaged,cross_signal_averaged,depo_averaged_raw
count,1.1985e+05,1.1985e+05,8.6517e+05,1.6864e+06,1.177e+05,1.177e+05,1.5089e+05,19019,19019
mean,6.8737e-06,-0.58196,0.63624,1.0136,1.0138,0.081916,1.0136,1.0078,0.063695
std,1.2718e-05,0.38772,0.31674,0.14362,0.032154,0.16467,0.13009,0.022717,0.2433
min,5.3393e-08,-9.8493,0.0037261,0.99282,0.99466,-1.2039,0.99763,0.99854,-1.3959
25%,5.5188e-07,-0.7529,0.44862,0.99921,1.0012,0.043945,0.99978,1.0001,0.026699
50%,2.072e-06,-0.5618,0.65611,1.0002,1.0046,0.068785,1.0001,1.0007,0.065753
75%,6.7244e-06,-0.4089,0.87974,1.0013,1.0153,0.11592,1.0004,1.0064,0.12219
95%,3.23e-05,-0.1031,1.1116,1.0182,1.0555,0.30471,1.0206,1.0396,0.44668
max,0.00019629,8.6111,1.1819,16.162,2.0177,27.307,12.348,1.7416,2.0797
Missing values,1.5666e+06,1.5666e+06,8.2127e+05,0,1.5687e+06,1.5687e+06,0,1.3187e+05,1.3187e+05


In [ ]:
%matplotlib qt
fig, ax = plt.subplots(3, 1, sharex=True, figsize=(24, 12))
ax = ax.flatten()
for i, name in enumerate(['depo_raw', 'beta_raw', 'v_raw']):
    p = ax[i].pcolormesh(df.data['time'],
                         df.data['range'],
                         df.data[name].transpose() if name != 'beta_raw' else np.log10(
                             df.data[name].transpose()),
                         cmap='jet', vmin=df.cbar_lim[name][0], vmax=df.cbar_lim[name][1])
    ax[i].set_title(name)
    fig.colorbar(p, ax=ax[i])
area = hd.area_select(df.data['time'],
                      df.data['range'],
                      df.data['depo_raw'].transpose(),
                      ax[0],
                      type=None)
plt.tight_layout()

In [ ]:
i = -1
final_result = pd.DataFrame(columns=['time', 'range', 'SNR', 'depo'])

In [ ]:
i += 1
%matplotlib qt
fig, ax = plt.subplots(figsize=(8, 5))
area_value = area.area[:, i]
area_range = df.data['range'][area.maskrange]
area_snr = df.data['co_signal'].transpose()[area.mask][:, i]
p = ax.scatter(area_value, area_range,
               c=area_snr,
               s=area_snr * 20)
ax.set_title(f"Depo value colored by SNR at time {df.data['time'][area.masktime][i]:.3f}")
ax.set_xlabel('Depo value')
ax.set_ylabel('Range')
fig.colorbar(p)



In [ ]:
max_i = np.argmax(area_snr)
result = pd.Series({'time': df.data['time'][area.masktime][i],
                    'range': area_range[max_i],
                    'SNR': area_snr[max_i],
                    'depo': area_value[max_i]})
result
final_result = final_result.append(result, ignore_index=True)
final_result


In [ ]:
plt.plot(final_result['depo'], '-')


In [ ]:
# Location etc
name = [int(df.more_info.get(key)) if key != 'location' else
        df.more_info.get(key).decode("utf-8") for
        key in ['location', 'year', 'month', 'day', 'systemID']]

filename = '-'.join([str(elem) for elem in name])

final_result.to_csv(filename + '.csv')


In [27]:

%matplotlib qt
fig, ax = plt.subplots(2, 1, sharex=False, figsize=(24, 12))
p = ax[0].pcolormesh(df.data['time'],
                     df.data['range'],
                     df.data['depo_raw'].transpose(),
                     cmap='jet', vmin=df.cbar_lim['depo_raw'][0],
                     vmax=df.cbar_lim['depo_raw'][1])
me = fig.colorbar(p, ax=ax[0])

area = hd.area_select(df.data['time'],
                      df.data['range'],
                      df.data['depo_raw'].transpose(),
                      ax[0],
                      ax[1],
                      type='time_point',
                      ref=df.data['co_signal'])

final_result = pd.DataFrame(columns=['time', 'range', 'SNR', 'depo'])

Chosen 48240 values


In [29]:
i = area.i
area_value = area.area[:, i]
area_range = df.data['range'][area.maskrange]
area_snr = df.data['co_signal'].transpose()[area.mask][:, i]

max_i = np.argmax(area_snr)
result = pd.Series({'time': df.data['time'][area.masktime][i],
                    'range': area_range[max_i],
                    'SNR': area_snr[max_i],
                    'depo': area_value[max_i]})
result
final_result = final_result.append(result, ignore_index=True)
final_result


,time,range,SNR,depo
0,5.4208,2295,1.0031,nan
1,5.4478,1575,1.0042,-0.12612


In [32]:
plt.plot(final_result['depo'], '-')


In [34]:
# Location etc
name = [int(df.more_info.get(key)) if key != 'location' else
        df.more_info.get(key).decode("utf-8") for
        key in ['location', 'year', 'month', 'day', 'systemID']]

filename = '-'.join([str(elem) for elem in name])

final_result.to_csv(filename + '.csv')